In [1]:
from utils.data import get_data_iterator
import torch

from torch.utils.data.dataloader import DataLoader
from utils.objaverse_dataset import ObjaversePointCloudDataset

annotations_file = '/Users/kostyalbalint/Documents/Egyetem/7.Felev/Szakdolgozat/objaverse_labeling/concatenated_annotations.npy'
pc_dir = '/Users/kostyalbalint/Documents/Egyetem/7.Felev/Szakdolgozat/pointClouds3000'

dataset = ObjaversePointCloudDataset(annotations_file=annotations_file, split=None, pc_dir=pc_dir, file_ext='.npz', scale_mode='unit_sphere', load_to_mem=False, transform=None)

val_loader = DataLoader(dataset, batch_size=1, num_workers=0)
data_iter = iter(val_loader)

In [34]:
import torch

from utils.misc import CheckpointManager
from models.autoencoder import AutoEncoder

dir = 'logs_ae/AE_2023_11_14__19_34_45_startfrompretrained'
device = 'mps'

ckpt_mgr = CheckpointManager(dir)

ckpt = ckpt_mgr.load_latest(map_location=device)

ckpt = torch.load('logs_ae/ckpt_9401915.000000_53000.pt', map_location=device)

model = AutoEncoder(ckpt['args'], size='big').to(device)
model.load_state_dict(ckpt['state_dict'])


<All keys matched successfully>

In [44]:
import plotly.graph_objects as go

def show_point_cloud(points, color, title):
    
    # Create a 3D scatter plot
    fig = go.Figure()
    
    # Scatter plot of the points
    fig.add_trace(go.Scatter3d(
        x=points[:, 0],
        y=points[:, 1],
        z=points[:, 2],
        mode='markers',
        marker=dict(
            size=1,
            color=color,  # You can customize the color here
        ),
    ))
    
    # Set layout options
    fig.update_layout(scene=dict(
        xaxis_title='X-axis',
        yaxis_title='Y-axis',
        zaxis_title='Z-axis',
        #aspectratio=dict(x=1, y=1, z=1),
        aspectmode='data'
        ),
        title=title,)
    
    # Show the plot
    return fig.show()

def show(i):
    og_model = x.cpu()[i]
    recon_model = recons.cpu()[i]
    show_point_cloud(recon_model, 'orange', 'Reconstructed')
    show_point_cloud(og_model, 'blue', 'Original')

In [112]:
def scale_pc(scale_mode, pc):
    if scale_mode == 'shape_unit':
        shift = pc.mean(dim=0).reshape(1, 3)
        scale = pc.flatten().std().reshape(1, 1)
    elif scale_mode == 'shape_half':
        shift = pc.mean(dim=0).reshape(1, 3)
        scale = pc.flatten().std().reshape(1, 1) / (0.5)
    elif scale_mode == 'shape_34':
        shift = pc.mean(dim=0).reshape(1, 3)
        scale = pc.flatten().std().reshape(1, 1) / (0.75)
    elif scale_mode == 'shape_bbox':
        pc_max, _ = pc.max(dim=0, keepdim=True)  # (1, 3)
        pc_min, _ = pc.min(dim=0, keepdim=True)  # (1, 3)
        shift = ((pc_min + pc_max) / 2).view(1, 3)
        scale = (pc_max - pc_min).max().reshape(1, 1) / 2
    elif scale_mode == 'unit_sphere':
        # Shift to the center of the point cloud
        shift = pc.mean(dim=0).reshape(1, 3)

        # Scale to fit into a unit sphere
        max_dist = torch.sqrt(((pc - shift) ** 2).sum(dim=1)).max()
        scale =  max_dist / torch.ones([1, 1])
    else:
        shift = torch.zeros([1, 3])
        scale = torch.ones([1, 1])

    return (pc - shift) / scale, shift, scale

In [13]:
og_pc = dataset[14]['pointcloud']

pc, s, sc = scale_pc('unit_sphere', og_pc)
print(sc)

show_point_cloud(pc, 'purple', 'Scale Testd')
show_point_cloud(og_pc, 'blue', 'Original')

NameError: name 'scale_pc' is not defined

In [46]:
import random

name_filter = 'Human'
filtered_annotations = [annotation for annotation in dataset.annotations.values() if
                        name_filter in annotation['name'] or name_filter in annotation['custom_label']]

id = dataset.uids.index(random.choice(filtered_annotations)['uid'])

id = dataset.uids.index('f25a052be431440fad238bcf736122e5')

with torch.no_grad():
    x = dataset[id]['pointcloud'][None].to(device)
    model.eval()
    code = model.encode(x)
    recons = model.decode(code, x.size(1), flexibility=0.0).detach()

for i in range(len(recons)):
    show(i)